# TF-IDF

We'll rely on term frequency times inverted document frequency to measure meaningful similarity between documents. Let's start by generating a matrix for the separate constituent parts of _Stjórn_.

In [1]:
import os,glob,json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def normalize(target):
    # This dict further standardizes the text beyond the rule set of stjorn-extract.ipynb:
    matrix = {
        'j': 'i',
        'v': 'u',
        'ð': 'þ',
        'á': 'a',
        'ǽ': 'æ',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ý': 'y',
        'ǿ': 'ø',
        'k': 'c',
        '[': '',
        ']': ''
        }
    for k,v in matrix.items():
        target = target.replace(k, v)
    return target

titles = ['prologue', 'introduction', 'gn', 'ex', 'lv', 'nm', 'dt', 'ios', 'idc', 'rt', '1sm', '2sm', '3rg', '4rg']
tokens = []
for title in titles:
    with open(f"nlp/{title}.txt") as raw:
        document = raw.read().replace('\n', ' ')
        tokens.extend(document.split())

work_indices = {
    'stjorn1': (650,124417),
    'stjorn2': (124417,147678),
    'stjorn3': (147678,156943,160719),
    'stjorn4': (156943,160719)
}

stjorn = dict()
for _work, _range in work_indices.items():
    if len(_range) == 2:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]]))
    else:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]] + tokens[_range[2]:]))

menota = dict()
for text in glob.glob('../menota/dipl/*txt'):
    ref = os.path.basename(text).replace('.txt', '')
    with open(text) as doc:
        # We'll subject Menota to the same normalization standard as Stjórn:
        menota[ref] = normalize(doc.read().replace('\n', ''))

In [3]:
vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(stjorn.values())
df = pd.DataFrame(cosine_similarity(model), stjorn.keys(), stjorn.keys())
df

,stjorn1,stjorn2,stjorn3,stjorn4
stjorn1,1.000000,0.811733,0.903770,0.842752
stjorn2,0.811733,1.000000,0.797618,0.833766
stjorn3,0.903770,0.797618,1.000000,0.835339
stjorn4,0.842752,0.833766,0.835339,1.000000


After eliminating vowel length and the þ/ð distinction, these are now all pretty similar to one another, with the biggest difference between _Stjórn II_ and _III_.

Now let's first add _Konungs skuggsjá_ from Menota, as well as Unger's own edition of the _Norwegian Homily Book_. Fingers crossed that we have got the normalization standard of the former to approach Unger's methods reasonably well.

In [4]:
# We want only those parts of Unger's NHB matched in Menota:
nhb_titles = ['alcuin', 'hom', 'olafr', 'visio', 'paternoster', 'anhang1']
nhb = ''
for title in nhb_titles:
    filepath = f'../nhb/nlp/{title}.txt'
    with open(filepath) as doc:
        nhb = nhb + normalize(doc.read().replace('\n', ''))
stjorn_plus = []
for v in stjorn.values():
    stjorn_plus.append(v)
stjorn_plus.extend([menota['nks235g_konungs_skuggsja'], nhb])
model = vectorizer.fit_transform(stjorn_plus)
df = pd.DataFrame(cosine_similarity(model), list(stjorn.keys()) + ['ks', 'nhb'], list(stjorn.keys()) + ['ks', 'nhb'])
df

,stjorn1,stjorn2,stjorn3,stjorn4,ks,nhb
stjorn1,1.000000,0.794809,0.889323,0.812473,0.714273,0.755032
stjorn2,0.794809,1.000000,0.779844,0.803534,0.700278,0.713765
stjorn3,0.889323,0.779844,1.000000,0.803896,0.741964,0.830661
stjorn4,0.812473,0.803534,0.803896,1.000000,0.665862,0.723180
ks,0.714273,0.700278,0.741964,0.665862,1.000000,0.754185
nhb,0.755032,0.713765,0.830661,0.723180,0.754185,1.000000


_Stjórn III_ and _Konungs skuggsjá_ share material cognate within the vernacular, so this connection standing out as the strongest between _Konungs skuggsjá_ and the constituent parts of _Stjórn_ comes as no surprise; if anything, the difference in match with the other parts is rather small. In fact, the _Norwegian Homily Book_ has a higher match with _Stjórn III_ than _Konungs skuggsjá_ does.

Next, let's model all of Menota along with Stjórn. Perhaps we'll leave Unger's _Homily Book_ in alongside the Menota edition, just for comparison's sake.

In [5]:
corpus = []
titles = []
for k,v in stjorn.items():
    titles.append(k)
    corpus.append(v)
titles.append('nhb')
corpus.append(nhb)
for k,v in menota.items():
    titles.append(k)
    corpus.append(v)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df

,stjorn1,stjorn2,stjorn3,stjorn4,nhb,nraNorrFragm75_kross_saga,am132_egils_saga,am162btheta_njals_saga,nraNorrFragm64_barlaams_saga,nraNorrFragm81A_benedikts_regla,am1056IX_konungs_skuggsja_fragment,am78_kristinrettir,am63_heimskringla3,dg4-7_strengleikar,am132_droplaugasona_saga,am132_kormaks_saga,nraNorrFragm72x76_dialogar,nraNorrFragm53_haralds_saga_hardrada,am132_finnboga_saga,nraNorrFragm70_agotu_saga,nraNorrFragm62_karlamagnuss_saga,nraNorrFragm60A_stjorn,am132_fostbraedra_saga,lbsFragm82_olafs_saga_helga,nraNorrFragm58B_konungs_skuggsja,nraNorrFragm60C_stjorn,holmPerg30_landslog,am619_norwegian_homily_book,nhb_am619,nraNorrFragm57_jons_saga_helga,nraNorrFragm69_nikulass_saga,am56_landslog,wolfAug9-10_egils_saga,nraNorrFragm66_thomass_saga,holmPerg17_thomass_saga,am383I_thorlaks_saga,holmPerg4_thidreks_saga,am132_njals_saga,am36_heimskringla2,am544_voluspa,am162bkappa_njals_saga,am305_landslog,nraNorrFragm58C_konungs_skuggsja,am132_olkofra_thattr,konungs_skuggsja_am243ba,nraNorrFragm54_sverris_saga,nraNorrFragm55B_hakonar_saga,nraNorrFragm79_mariu_saga,gks2365_voluspa,am243balpha_konungs_skuggsja,nraNorrFragm51_fagrskinna,am132_viga-glums_saga,am279a_gragas,am677_gregory,am132_laxdoela_saga,am302_landslog,am178_thidreks_saga,nraNorrFragm81B_benedikts_regla,am132_bandamanna_saga,nraNorrFragm71_gregors_saga_pafa,gregory_am677,am655_laeknisbok,am519a_alexanders_saga,holmPerg34_landslog,am162balpha_njals_saga,am113_islendingabok,nraNorrFragm7_landslog,nraNorrFragm67_thomass_saga,nraNorrFragm56_thorgils_saga,nks235g_konungs_skuggsja,am132_hallfredar_saga,am35_heimskringla1,am242_codex_wormianus,nraNorrFragm78_mariu_saga,konungs_skuggsja_fragment_am1056xi,dg8II_olafs_saga,nraNorrFragm80_pals_saga,nraNorrFragm63_karlamagnuss_saga,nraNorrFragm77_dialogar,am28_codex_runicus,holmPerg34_boejarlog,dg8I_landslog,nraNorrFragm60B_stjorn,nraNorrFragm55A_hakonar_saga,skbA120_marys_complaint,nraNorrFragm59_rimbegla,nraNorrFragm65_floress_saga,nraNorrFragm52_olafs_saga_helga_hin_elzta,holmPerg6_barlaams_saga,nraNorrFragm68_brendanuss_saga,nraNorrFragm61_karlamagnuss_saga,nraNorrFragm58A_konungs_skuggsja
stjorn1,1.000000,0.675387,0.783339,0.649742,0.643941,0.257274,0.672527,0.571604,0.500302,0.222923,0.368637,0.421182,0.651110,0.728837,0.581742,0.476610,0.494489,0.391262,0.575325,0.233495,0.598066,0.357409,0.627533,0.573677,0.473834,0.484881,0.362040,0.632119,0.632119,0.605251,0.623642,0.424192,0.652068,0.602263,0.628994,0.424009,0.690261,0.676023,0.671307,0.310597,0.519099,0.413216,0.561085,0.506557,0.555367,0.668246,0.431973,0.444954,0.328881,0.555367,0.386150,0.655450,0.341240,0.512100,0.661238,0.401987,0.149346,0.356428,0.600877,0.367890,0.512100,0.456324,0.663204,0.554184,0.497871,0.367190,0.425358,0.669585,0.465424,0.545661,0.552140,0.664571,0.710394,0.638211,0.368637,0.618251,0.610497,0.641380,0.593380,0.227246,0.520304,0.470037,0.293207,0.512047,0.195796,0.432415,0.453895,0.606203,0.726365,0.488334,0.485173,0.583016
stjorn2,0.675387,1.000000,0.654300,0.628096,0.573106,0.252889,0.604286,0.493187,0.458971,0.198467,0.325867,0.413019,0.568860,0.627843,0.540389,0.438563,0.442589,0.343132,0.500694,0.214664,0.544635,0.280767,0.564768,0.507675,0.431108,0.391072,0.373915,0.575543,0.575543,0.529738,0.518355,0.411399,0.592197,0.488766,0.463867,0.341887,0.642173,0.622512,0.578432,0.285286,0.485023,0.399338,0.527731,0.482388,0.504161,0.616213,0.404941,0.391499,0.312157,0.504161,0.358301,0.596529,0.313370,0.478295,0.598553,0.389952,0.171318,0.325514,0.558256,0.347516,0.478295,0.425943,0.578361,0.563611,0.478745,0.309535,0.412559,0.579681,0.382934,0.507153,0.515963,0.574245,0.608711,0.559493,0.325867,0.557188,0.541039,0.603547,0.533864,0.197462,0.541053,0.464794,0.280207,0.434213,0.165643,0.412782,0.421810,0.557772,0.668520,0.444856,0.457296,0.539522
stjorn3,0.783339,0.654300,1.000000,0.617894,0.762637,0.283704,0.783070,0.650753,0.558589,0.266439,0.404821,0.492708,0.739716,0.817489,0.672118,0.550083,0.563120,0.476378,0.649573,0.274033,0.686795,0.3

The score for the two editions of the _Norwegian Homily Book_ may serve as our proof of method: these come to 0.97 similarity. As this compares an edition of Unger's with a Menota transcription, as does our comparison of _Stjórn_ with the remainder of the Menota corpus, we may be confident that the scores give a fair indication of lexical similarity.